In [ ]:
import pandas as pd
import sys 
from matplotlib import pyplot as plt
import os
%matplotlib inline
sys.path.append('../')
from src.plots import add_panel_text
import numpy as np

from src.constants import *
OUTPUT_DIR = '/app/output'
DATA_DIR = '/app/data/mimic-iv-2.0/'

# Load Data

In [ ]:
patients_file = os.path.join(DATA_DIR, 'hosp', 'patients.csv.gz')
admissions_file = os.path.join(DATA_DIR, 'hosp', 'admissions.csv.gz')
lab_file = os.path.join(DATA_DIR, 'hosp', 'labevents.csv.gz')
lab_meta_file = os.path.join(DATA_DIR, 'hosp', 'd_labitems.csv.gz')

In [ ]:
patients_df = pd.read_csv(patients_file, compression='gzip')
patients_df.head()

In [ ]:
print(len(patients_df))

In [ ]:
# fig, ax = plt.subplots(1,1,dpi=100)
# tmp = patients_df[[AGE_COL, GENDER_COL]]
# tmp[AGE_COL] = pd.cut(tmp[AGE_COL], bins=AGE_BINS, labels=AGE_LABELS)
# tmp.groupby([AGE_COL, GENDER_COL]).size().unstack().plot(kind='bar', ax=ax)
# ax.set_xlabel('Anchor Age [years]', fontsize=font_sz)
# ax.set_ylabel('Number of Patients', fontsize=font_sz)
# ax.set_title(f'Total Population, N={len(tmp)}', fontsize=font_sz)
# ax.legend(labels=['Female', 'Male'], title="Sex")
# ax.set_xticklabels(AGE_LABELS, rotation=90)
# fig.savefig(os.path.join(OUTPUT_DIR, 'age_gender_total.png'), dpi=300)

In [ ]:
# fig, ax = plt.subplots(1,1,dpi=100)
# patients_df[YEAR_GROUP_COL].value_counts().plot.bar(ax=ax)
# ax.set_ylabel('Number of Patients', fontsize=font_sz)
# ax.set_xlabel('Anchor Year Group', fontsize=font_sz)
# for p in ax.patches:
#     ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
admissions_df = pd.read_csv(admissions_file, compression='gzip', parse_dates=[ADMISSION_TIME_COL,
                            DISCHARGE_TIME_COL, DEATH_TIME_COL, ED_REG_TIME, ED_OUT_TIME])
admissions_df

In [ ]:
admissions_df = admissions_df.merge(patients_df, on=[SUBJECT_ID_COL])

# Calculate Age at Admission and Group of Admission Year

Based on mimic IV example https://mimic.mit.edu/docs/iv/modules/hosp/patients/

In [ ]:
# Diff column first
admissions_df[ADMISSION_YEAR_COL] = (admissions_df[ADMISSION_TIME_COL].dt.year - admissions_df['anchor_year'])

# Age at admission calculation
admissions_df[ADMISSION_AGE_COL] = (admissions_df[AGE_COL] + admissions_df[ADMISSION_YEAR_COL])

# Admission year group lower bound calculation
admissions_df[ADMISSION_YEAR_COL] = admissions_df[ADMISSION_YEAR_COL] + admissions_df[YEAR_GROUP_COL].apply(lambda x: int(x.split(' ')[0]))


In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[ADMISSION_YEAR_COL].value_counts().sort_index().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Admission Year (lower bound)', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
tmp = admissions_df[[ADMISSION_AGE_COL, GENDER_COL]]
tmp[ADMISSION_AGE_COL] = pd.cut(tmp[ADMISSION_AGE_COL], bins=AGE_BINS, labels=AGE_LABELS)
tmp.groupby([ADMISSION_AGE_COL, GENDER_COL]).size().unstack().plot(kind='bar', ax=ax)
ax.set_xlabel('Age at Admission [years]', fontsize=font_sz)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_title(f'Total Population, N={len(tmp)}', fontsize=font_sz)
ax.legend(labels=['Female', 'Male'], title="Sex")
ax.set_xticklabels(AGE_LABELS, rotation=90)
fig.savefig(os.path.join(OUTPUT_DIR, 'age_gender_admissions_total.png'), dpi=300)

In [ ]:
admissions_df[LOS_EXACT_COL] = (admissions_df[DISCHARGE_TIME_COL] - admissions_df[ADMISSION_TIME_COL])
admissions_df[LOS_DAYS_COL] = admissions_df[LOS_EXACT_COL].dt.ceil('1d')

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[ADMISSION_TYPE_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Admission Type', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
max_clip_days = 14

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

ax = axes[0]
tmp = admissions_df[admissions_df[ADMISSION_TYPE_COL] == 'URGENT']
los_bar = tmp[LOS_DAYS_COL].clip(pd.to_timedelta('1d'), pd.to_timedelta(f'{max_clip_days}d')).value_counts().sort_index()
los_bar.index = np.arange(1, max_clip_days+1)
los_bar.plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('LOS (Days)', fontsize=font_sz)
ax.grid(axis='y')
ax.set_title('URGENT', fontsize=font_sz)

ax = axes[1]
tmp = admissions_df[admissions_df[ADMISSION_TYPE_COL] == 'EW EMER.']
los_bar = tmp[LOS_DAYS_COL].clip(pd.to_timedelta('1d'), pd.to_timedelta(f'{max_clip_days}d')).value_counts().sort_index()
los_bar.index = np.arange(1, max_clip_days+1)
los_bar.plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('LOS (Days)', fontsize=font_sz)
ax.grid(axis='y')
ax.set_title('EW EMER.', fontsize=font_sz)

ax = axes[2]
tmp = admissions_df[admissions_df[ADMISSION_TYPE_COL] == 'DIRECT EMER.']
los_bar = tmp[LOS_DAYS_COL].clip(pd.to_timedelta('1d'), pd.to_timedelta(f'{max_clip_days}d')).value_counts().sort_index()
los_bar.index = np.arange(1, max_clip_days+1)
los_bar.plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('LOS (Days)', fontsize=font_sz)
ax.grid(axis='y')
ax.set_title('DIRECT EMER.', fontsize=font_sz)

fig.tight_layout()

# Taking only URGENT admissions from now on

In [ ]:
print(len(admissions_df))
admissions_df = admissions_df[admissions_df[ADMISSION_TYPE_COL] == 'URGENT']
print(len(admissions_df))

# Counting urgent admissions to each patient 

In [ ]:
number_of_admissions = admissions_df.groupby(SUBJECT_ID_COL)[ADMISSION_ID_COL].nunique()
number_of_admissions.name = ADMISSION_COUNT_COL
number_of_admissions

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
number_of_admissions.value_counts().sort_index().plot.bar(ax=ax, logy=True)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Number of Admissions', fontsize=font_sz)
ax.grid('y', which='minor', alpha=0.4)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
admissions_df = admissions_df.merge(number_of_admissions, on=SUBJECT_ID_COL)
only_last_admission = admissions_df.sort_values(by=[ADMISSION_TIME_COL]).drop_duplicates(subset=[SUBJECT_ID_COL], keep='last')
len(only_last_admission)

# Taking only patients with last admission after MINIMUM YEAR

In [ ]:
MINIMUM_YEAR = 2017
print(len(only_last_admission))
only_last_admission = only_last_admission[only_last_admission[ADMISSION_YEAR_COL] >= MINIMUM_YEAR]
print(len(only_last_admission))

In [ ]:
admissions_df[admissions_df[SUBJECT_ID_COL] == 13465885].sort_values(ADMISSION_TIME_COL)

In [ ]:
admissions_df[admissions_df[SUBJECT_ID_COL] == 17772216].sort_values(ADMISSION_TIME_COL)

In [ ]:
pids = only_last_admission[SUBJECT_ID_COL].drop_duplicates()
adm_ids = only_last_admission[ADMISSION_ID_COL].drop_duplicates()
print(len(pids))
print(len(adm_ids))

# Creating full_df for patients with urgent admissions after minimum year and with lab tests

In [ ]:
chunksize = 10 ** 7
full_df = pd.DataFrame()
with pd.read_csv(lab_file, chunksize=chunksize, compression='gzip', parse_dates=[CHART_TIME_COL, STORE_TIME_COL]) as reader:
    for chunk in reader:
        tmp_chunk = chunk[chunk[SUBJECT_ID_COL].isin(pids) & chunk[ADMISSION_ID_COL].isin(adm_ids)]
        tmp_adms = only_last_admission[only_last_admission[SUBJECT_ID_COL].isin(pids) & only_last_admission[ADMISSION_ID_COL].isin(adm_ids)]
        tmp_patinets = patients_df[patients_df[SUBJECT_ID_COL].isin(pids)]
        tmp_chunk = tmp_chunk.merge(tmp_adms, on=[SUBJECT_ID_COL, ADMISSION_ID_COL])
        tmp = tmp_chunk.merge(tmp_patinets, on=[SUBJECT_ID_COL])
        full_df = pd.concat([full_df, tmp])
        print(len(full_df))

full_df.head()

# Continue only with included patients_df and admissions_df

In [ ]:
pids = full_df[SUBJECT_ID_COL].drop_duplicates().values
adms_ids = full_df[ADMISSION_ID_COL].drop_duplicates().values
print(len(patients_df))
patients_df = patients_df[patients_df[SUBJECT_ID_COL].isin(pids)]
print(len(patients_df))
print(len(admissions_df))
admissions_df = admissions_df[admissions_df[ADMISSION_ID_COL].isin(adms_ids)]
print(len(admissions_df))

In [ ]:
len(full_df)

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[ADMISSION_LOCATION_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Admission Location', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[DISCHARGE_LOCATION_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Discharge Location', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

# Regrouping discharge location

In [ ]:
DISCHARGE_REGROUPING_DICT = {
    'HOME': 'HOME',
    'HOME HEALTH CARE': 'HOME',
    'SKILLED NURSING FACILITY': 'FURTHER TREATMENT',
    'DIED': 'DIED',
    'REHAB': 'HOME',
    'CHRONIC/LONG TERM ACUTE CARE': 'FURTHER TREATMENT',
    'HOSPICE': 'FURTHER TREATMENT',
    'AGAINST ADVICE': 'CENSORED',
    'ACUTE HOSPITAL': 'FURTHER TREATMENT',
    'PSYCH FACILITY': 'FURTHER TREATMENT',
    'OTHER FACILITY': 'FURTHER TREATMENT',
    'ASSISTED LIVING': 'HOME',
    'HEALTHCARE FACILITY': 'FURTHER TREATMENT',
}

In [ ]:
admissions_df[DISCHARGE_LOCATION_COL].replace(DISCHARGE_REGROUPING_DICT, inplace=True)
full_df[DISCHARGE_LOCATION_COL].replace(DISCHARGE_REGROUPING_DICT, inplace=True)

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[DISCHARGE_LOCATION_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Discharge Location', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
tmp = admissions_df[[ADMISSION_AGE_COL, GENDER_COL]]
tmp[ADMISSION_AGE_COL] = pd.cut(tmp[ADMISSION_AGE_COL], bins=AGE_BINS, labels=AGE_LABELS)
tmp.groupby([ADMISSION_AGE_COL, GENDER_COL]).size().unstack().plot(kind='bar', ax=ax)
ax.set_xlabel('Age at Admission [years]', fontsize=font_sz)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_title(f'Total Population, N={len(tmp)}', fontsize=font_sz)
ax.legend(labels=['Female', 'Male'], title="Sex")
ax.set_xticklabels(AGE_LABELS, rotation=90)
fig.savefig(os.path.join(OUTPUT_DIR, 'age_gender_admissions_subset.png'), dpi=300)

# Regroup Race

In [ ]:
RACE_REGROUPING_DICT = {
    'WHITE': 'WHITE',
    'UNKNOWN': 'OTHER',
    'BLACK/AFRICAN AMERICAN': 'BLACK',
    'OTHER': 'OTHER',
    'ASIAN': 'ASIAN',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC',
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC',
    'ASIAN - CHINESE': 'ASIAN',
    'BLACK/CARIBBEAN ISLAND': 'BLACK',
    'BLACK/AFRICAN': 'BLACK',
    'BLACK/CAPE VERDEAN': 'BLACK',
    'PATIENT DECLINED TO ANSWER': 'OTHER',
    'WHITE - BRAZILIAN': 'WHITE',
    'PORTUGUESE': 'HISPANIC', 
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'WHITE - RUSSIAN': 'WHITE',
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER',
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC',
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC',
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC',
    'SOUTH AMERICAN': 'HISPANIC',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC',
    'HISPANIC/LATINO - CUBAN': 'HISPANIC',
    'ASIAN - KOREAN': 'ASIAN',
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC',
    'UNABLE TO OBTAIN': 'OTHER',
    'HISPANIC OR LATINO': 'HISPANIC'
}

In [ ]:
admissions_df[RACE_COL].replace(RACE_REGROUPING_DICT, inplace=True)
full_df[RACE_COL].replace(RACE_REGROUPING_DICT, inplace=True)

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[RACE_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Race', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

In [ ]:
fig, ax = plt.subplots(1,1,dpi=100)
admissions_df[INSURANCE_COL].value_counts().plot.bar(ax=ax)
ax.set_ylabel('Number of Patients', fontsize=font_sz)
ax.set_xlabel('Insurance', fontsize=font_sz)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x(), p.get_height() * 1.01))

# Taking only results 24 hours from admission

In [ ]:
full_df.head()

In [ ]:
full_df[ADMISSION_TO_RESULT_COL] = (full_df[STORE_TIME_COL] - full_df[ADMISSION_TIME_COL])

In [ ]:
print(len(full_df[ADMISSION_TO_RESULT_COL]))
print(len(full_df[full_df[ADMISSION_TO_RESULT_COL] <= pd.to_timedelta('1d')]))
print(len(full_df[full_df[ADMISSION_TO_RESULT_COL] <= pd.to_timedelta('2d')]))

In [ ]:
print(len(full_df[ADMISSION_ID_COL].drop_duplicates()))
print(len(full_df[full_df[ADMISSION_TO_RESULT_COL] <= pd.to_timedelta('1d')][ADMISSION_ID_COL].drop_duplicates()))
print(len(full_df[full_df[ADMISSION_TO_RESULT_COL] <= pd.to_timedelta('2d')][ADMISSION_ID_COL].drop_duplicates()))

In [ ]:
full_df = full_df[full_df[ADMISSION_TO_RESULT_COL] <= pd.to_timedelta('1d')]
full_df.head()

In [ ]:
print(len(full_df))
full_df.sort_values(by=[ADMISSION_TIME_COL, STORE_TIME_COL]).drop_duplicates(subset=[SUBJECT_ID_COL, ADMISSION_ID_COL, ITEM_ID_COL], 
    inplace=True, keep='last')
print(len(full_df))

# Most common lab tests upon arrival

In [ ]:
lab_meta_df = pd.read_csv(lab_meta_file, compression='gzip')
lab_meta_df

In [ ]:
threshold = 4500

In [ ]:
common_tests = full_df.groupby(ITEM_ID_COL)[ADMISSION_ID_COL].nunique().sort_values(ascending=False)
included_in_threshold = common_tests[common_tests > threshold].to_frame().merge(lab_meta_df, on=ITEM_ID_COL)
included_in_threshold

In [ ]:
print(len(full_df))
full_df = full_df[full_df[ITEM_ID_COL].isin(included_in_threshold[ITEM_ID_COL].values)]
print(len(full_df))

In [ ]:
full_df.head()

In [ ]:
full_df.columns

In [ ]:
full_df['flag'].value_counts()

In [ ]:
# to include priority? to include "comments" flag?
# to select base on admission location?


In [ ]:
MARITAL_STATUS_COL = 'marital_status'

In [ ]:
COLUMNS_SUBSET = [SUBJECT_ID_COL, ITEM_ID_COL, 'flag', DISCHARGE_LOCATION_COL, INSURANCE_COL, MARITAL_STATUS_COL, RACE_COL, 'gender_x', ADMISSION_AGE_COL, LOS_DAYS_COL]
OUTPUT_COLS = [DISCHARGE_LOCATION_COL, LOS_DAYS_COL]

In [ ]:
tmp = full_df.copy()

In [ ]:
tmp

In [ ]:
tmp['flag'] = tmp['flag'].replace({'abnormal': 1}).fillna(0)
tmp

In [ ]:
tmp[[SUBJECT_ID_COL, ITEM_ID_COL, 'flag']].drop_duplicates(subset=[SUBJECT_ID_COL, ITEM_ID_COL]).fillna(0).pivot(index=SUBJECT_ID_COL, columns=ITEM_ID_COL, values='flag')
x

In [ ]:
tmp[[SUBJECT_ID_COL, ITEM_ID_COL, 'flag']]